## Fast Fourier Transform (FFT)
Este material trata dos pilares para cômputo eficiente da DFT.

<img src="./figures/butterfly_FFT.png" width="200"/>

A DFT de uma sequência de comprimento $N$ é
\begin{equation}\label{eq:DFT}
    X[k] = \sum_{n=0}^{N-1} x[n]W_{N}^{kn}, \qquad k=0,1,\ldots,N-1 
\end{equation}
onde $W_{N}\triangleq e^{-j(2\pi/N)}$.

A DFT inversa é dada por
\begin{equation}
    x[n] = \frac{1}{N}\sum_{k=0}^{N-1} X[k]W_{N}^{-kn}, \qquad n=0,1,\ldots,N-1 
\end{equation}

As duas sequências $x[n]$ e $X[k]$ podem ser de valor complexo, e para serem computadas precisam de multiplicações e somas complexas. A única diferença entre as duas é o fator de escala $1/N$ e o sinal do expoente $W_{N}$ (que é na verdade o conjugado complexo).  Assim o procedimento de cálculo da DFT ou DFT inversa seguem o mesmo procedimento computacional.

Observando a equação da DFT, no cômputo de $X[k]$ para um determinado valor de $k$ são necessárias $N$ multiplicações complexas e $N-1$ somas complexas.

Portanto, para os $N$ valores da sequência $X[k]$, são necessárias $N^2$ multiplicações complexas e $N(N-1)\approx N^2$ somas complexas. Assim, a complexidade computacional da DFT de uma sequência de $N$ valores é da ordem de $N^2$ operações.

Observações:
- 1 multiplicação complexa requer 4 multiplicações reais e 2 somas reais. Por outro lado, uma soma complexa exige 2 somas reais;
- Nesta conta o cálculo dos coeficientes $W_{N}^{kn}$ é desconsiderada, os valores estão armazenados em memória;
- Pode-se usar propriedades como
\begin{eqnarray}
 W_{N}^{kn} & = & 1 \qquad\qquad \text{para } n,k=0\\
 W_{N}^{k(N-n)} & = & W_{N}^{-kn} = \left(W_{N}^{kn}\right)^{\ast} \quad \text{(simetria conjugada)}\\
 W_{N}^{kn} & = & W_{N}^{k(N+n)} = W_{N}^{(k+N)n} \quad \text{(periodicidade)}
\end{eqnarray}

Usando a primeira identidade é possível eliminar algumas multiplicações. Usando a simetria complexa conjugada o número de multiplicações pode ser reduzido aproximadamente por um fator de $2$. Explorando estas duas e mais a propriedade de periodicidade é possível reduzir um pouco mais a complexidade do cômputo da DFT, mas a ordem continua proporcional a $N^2$.

Em 1965 a ordem de complexidade do cômputo da DFT foi drasticamente reduzida para $N\log_{2}(N)$ quando James Cooley and John Tukey publicaram o trabalho intitulado ["An algorithm for the Machine Calculation of Complex Fourier Series"](https://www.ams.org/journals/mcom/1965-19-090/S0025-5718-1965-0178586-1/). A família de algoritmos para cômputo eficiente da DFT que emergiu desta proposta foi denomina <i>fast Fourier transform</i> (FFT).

<b>Os algoritmos FFT são baseados na princípio de decompor o cômputo da DFT de uma sequência de comprimento N sucessivamente em DFTs menores</b>. A maneira como este princípio é implementado leva à variedade de algoritmos, todos com drástica melhora no tempo de cômputo.

Existem duas grandes famílias dos algoritmos FFT: <b>dizimação no tempo</b> e <b>dizimação na frequência</b>. Vamos tratar do algoritmo original de Cooley e Tukey, conhecido como Radix-2 DIT (<i>decimation in time</i>).

## FFT por dizimação no tempo
Suponha que a sequência $x[n]$ tem comprimento $N$ e este número é potência de 2 (ou seja, $N=2^l$).

A motivação está no fato de podermos dividir o cômputo da DFT em duas componentes de forma sucessiva até termos apenas cômputo de DFTs de tamanho $N=2$ amostras. E o interessante nisto é que DFTs com $N=2$ não demandam multiplicações. Veja o seguinte:
\begin{equation}
    X[k] = \sum_{n=0}^{N-1} x[n]W_{N}^{kn} = \sum_{n=0}^{1} x[n]W_{2}^{kn}, \qquad k=0,1 
\end{equation}
Mas
\begin{equation}
  W_{2}^{kn} = e^{-j(\frac{2\pi kn}{N})} = \begin{array}{rl}
   1, & \text{$n$ ou $k$}=0 \\
   -1, & n=k=1\end{array}
\end{equation}
Logo,
\begin{eqnarray}
    X[0] & = & x[0]+x[1]\,,\\
    X[1] & = & x[0]-x[1]\,.
\end{eqnarray}

Este cálculo pode ser representado pelo fluxograma da figura a seguir, denominado borboleta de duas entradas:

<img src="./figures/borboleta_x0x1.png" width="45%"/>
e utilizando esta estrutura os algoritmos FFT alcançam a sua eficiência, computando a DFT de $N$ pontos de maneira exata.

Para ver melhor como ocorre esta construção (ou decomposição em DFTs sucessivas), como $N$ é um inteiro par, vamos dividir o cômputo por 2 separando inicialmente os índices pares e ímpares:
\begin{equation}
 X[k] = \sum_{n \text{ par}} x[n]W_{N}^{kn} + \sum_{n \text{ ímpar}} x[n]W_{N}^{kn}
\end{equation}
que com a substituição das variáveis $n=2r$ para $n$ par e $n=2r+1$ para $n$ ímpar
\begin{equation}
 X[k] = \sum_{r=0}^{\frac{N}{2}-1} x[2r]W_{N}^{2rk} + \sum_{r=0}^{\frac{N}{2}-1} x[2r+1]W_{N}^{(2r+1)k}
\end{equation}

Observe que $W_{N}^{2rk}=W_{N/2}^{rk}$ pois $W_{N}^{2rk} = e^{-j2\pi 2rk/N} = e^{-j2\pi rk/(N/2)}$, e portanto podemos reescrever
\begin{equation}
 X[k] = \sum_{r=0}^{\frac{N}{2}-1} x[2r]W_{\frac{N}{2}}^{rk} + W_{N}^{k}\sum_{r=0}^{\frac{N}{2}-1} x[2r+1]W_{\frac{N}{2}}^{rk}
\end{equation}

Definindo as sequências
\begin{equation}
 a^{N/2}[r]=x^{N}[2r] \quad\text{e}\quad b^{N/2}[r] = x^{N}[2r+1]
\end{equation}
temos
\begin{equation}
 X[k] = \sum_{r=0}^{\frac{N}{2}-1} a^{N/2}[r]W_{\frac{N}{2}}^{rk} + W_{N}^{k}\sum_{r=0}^{\frac{N}{2}-1} b^{N/2}[r]W_{\frac{N}{2}}^{rk}
\end{equation}
e observando que as DFTs das sequências $a^{N/2}[r]$ e $b^{N/2}[r]$ são dadas por
\begin{equation}
    A[k] = \sum_{r=0}^{\frac{N}{2}-1} a^{N/2}[r]W_{\frac{N}{2}}^{kn}, \qquad k=0,1,\ldots,\frac{N}{2}-1 
\end{equation}
\begin{equation}
    B[k] = \sum_{r=0}^{\frac{N}{2}-1} b^{N/2}[r]W_{\frac{N}{2}}^{kn}, \qquad k=0,1,\ldots,\frac{N}{2}-1 
\end{equation}

Podemos escrever
\begin{equation}
X[k] = A[k] + W_{N}^{kn}B[k], \qquad k=0,1,\ldots,N-1
\end{equation}

e esta descrição faz o cálculo da DFT de $N$ pontos como duas DFTs de $N/2$ pontos, uma dos índices pares e a outra dos índices ímpares da sequência original $x[n]$.

Veja a seguir o fluxograma para o cálculo da DFT de $N=8$ pontos, usando duas transformadas de $N/2=4$ pontos e as borboletas de duas entradas.

<img src="./figures/FFT8ptos_DFT4ptos.png" width="55%"/>

As saídas das transformadas de $4$ pontos são combinadas através das borboletas para compor a saída do cômputo da DFT.

Cada uma destas transformadas exige $(N/2)^2$ operações de multiplicação e soma. Além destas, precisamos das $N$ operações de soma das borboletas de saída, somando $2(N/2)^2+N$ operações.

Reaplicando o princípio da divisão por 2 novamente chegamos a duas transformadas de $N/4$ pontos para o cômputo de uma DFT de $N/2$ pontos. Neste exemplo em que $N=8$ com esta nova divisão já chegamos ao máximo da decomposição por dizimação no tempo, como ilustrado na figura a seguir

<img src="./figures/FFT8ptos_DFT2ptos.png" width="70%"/>

Nesta estrutura já temos duas camadas de borboletas. Substituindo agora o cômputo das DFTs de 2 pontos por borboletas, vemos que o cômputo da DFT de $N$ pontos pode ser feito inteiramente com células básicas de duas entradas.

<img src="./figures/FFT8ptos_borboletas.png" width="75%"/>

Temos ainda os coeficientes multiplicativos $W_{N}^{kn}$ e $W_{N}^{k+N/2}$. Mas pela propriedade da periodicidade temos que $W_{N}^{N/2}=-1$, e portanto
\begin{equation}
    W_{N}^{k+N/2} = W_{N}^{N/2}.W_{N}^{k} = -W_{N}^{k}
\end{equation}
permitindo simplificar ainda mais a estrutura

<img src="./figures/FFT8ptos_final.png" width="75%"/>

Concluímos que temos $l=\log_{2}N$ estágios de cálculo, cada um com $N/2$ borboletas. Cada borboleta requer 1 multiplicação complexa e 2 somas complexas. Portanto, cada estágio requer $(N/2)$ multiplicações complexas e $N$ somas complexas. Logo, o cômputo do fluxograma exige $(N/2)\log_2(N)$ multiplicações e $N\log_2(N)$ somas complexas. A tabela a seguir mostra como a redução do número de operações pela FFT cresce com $N$.

<img src="./figures/tabela_complexidade_FFT.png" width="40%"/>

Existem ainda outras reduções que são exploradas, mas estas mostradas são as mais drásticas e importantes.

### Dica: Amostre rápido o suficiente e em bom tamanho

Quando digitalizamos sinais contínuos com ADC, sabemos que a taxa de amostragem deve ser maior ou igual a duas vezes a largura de banda do sinal para não termos sobreposição espectral (<i>aliasing</i>).

Uma boa prática é amostrar a uma taxa 2.5 a 4 vezes a largura de banda.

Se não sabemos a largura de banda do sinal, usamos a máxima taxa de amostragem do ADC e não confiamos nas componentes próximas à frequência máxima de representação, ou seja, $f_s/2$.

Se suspeitamos de <i>aliasing</i> nas componentes mais altas, ou há ruído de banda larga, usamos um filtro passa-baixas antes do ADC. A frequência de corte deve ser maior que a banda de frequência de interesse da aplicação, mas um tanto menor que metade da frequência de amostragem.

Outra questão fundamental:

<b>Quantas amostras devemos considerar para realizar o cálculo da FFT?</b> A resposta é que o comprimento deve ser tal que atenda a resolução de frequência pretendida no espectro calculados pela FFT, dada uma frequência de amostragem já definida.

De modo geral, quanto maior a resolução em frequência, melhor a qualidade da nossa análise. Assim, podemos definir o comprimento da sequência através da resolução mínima pretendida, $\Delta f = f_s/N$. Assim, o tempo total da coleta de dados é $N/f_s$ segundos. Por exemplo, se precisamos de uma resolução espectral de $5$ Hz, então
\begin{equation}
    N = \frac{f_s}{\Delta f} = \frac{f_s}{5} = 0.2 f_s
\end{equation}

Neste caso, se $f_s$ é $10$ kHz, então $N$ deve ser pelo menos $2000$, e escolheríamos $N=2048$ por ser o próximo em potência de 2.